# Redes Neuronales - TP1

## Ej 2

1. Comprobar estadísticamente la capacidad de la red de Hopfield ‘82 calculando la cantidad máxima de patrones pseudo-aleatorios aprendidos en función del tamaño de la red. Obtener experimentalmente los resultados de la siguiente tabla (los valores de la tabla corresponden a una iteración con actualización sincrónica).

|$P_{error}$|${p_{max}}/{N}$|
|-|-|
|0.001|0.105|
|0.0036|0.138|
|0.01|0.185|
|0.05|0.37|
|0.1|0.61|

2. Proponga una manera de generar patrones con distintos grados de correlación.
Utilice el método propuesto para analizar cómo varía la capacidad de la red de
Hopfield en función de la correlación entre patrones.


Mi idea es crear una función o clase que reciba un set de probabilidades de error y un tamaño de red particular, y devuelva la capacidad para cada umbral de error. 

En lo que respecta a patrones, siempre se va a evaluar con patrones con una correlación determinada. un método los va a generar aleatoriamente, otro debería poder hacerlos con una correlación "custom". 


Una aclaración importante: el primer inciso va a parecer desprolijo en comparación con este, pero es porque en ese prioricé tener algo funcionando. Ahora que entendí un poco más es más fácil colocar todo en una clase y operar desde ahí. 

Para el inciso que genera vectores correlacionados se hace lo siguiente. Se suponen vectores X e Y generados de tal forma que toman valores discretos $-1$ y $1$ con probabilidad $1/2$. Así, sus medias son cero y las varianzas son unitarias. Se plantea el coeficiente de correlación de pearson como:

$$
corr_{coef} = \frac{cov(X,Y)}{\sqrt{Var(X)\cdot Var(Y)}} =  E[X \cdot Y] - E[X]E[Y] = E[X \cdot Y]
$$

Y ahora $E[X \cdot Y]$ se abre por esperanza total en:

$$
E[X \cdot Y] = P(X=1)P(Y=1)(1)(1) * P(X=1)P(Y=-1)(1)(-1) * P(X=-1)P(Y=1)(-1)(1) * P(X=-1)P(Y=-1)(-1)(-1)
$$

Denomino $p = P(X=Y)$ la probabilidad de que los vectores concuerden en algún bit. 

$$
E[X \cdot Y] = p - (1-p) = 2p-1
$$

Entonces:
$$
corr_{coef} = 2p-1 \Leftrightarrow p = \frac{corr_{coef} +1}{2}
$$

De esta manera se puede tomar un vector X generado como se generaría cualquier vector aleatorio y luego se genera un Y en base a la probabilidad de que concuerde con el X. Si $x=1$ se tira una moneda de probabilidad p de que $y=x$. 

In [1]:
# primero importamos numpy y algo para leer imágenes y hacer graficos
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
class evaluacion_capacidad_red_neuronal:
    def __init__(self, cantidad_neuronas):
        self.N = cantidad_neuronas
        self.W = None

    def evaluar(self, datos_prueba):
        # Lógica para evaluar la capacidad del modelo
        pass

    def estado_aleatorio(self,correlacion):
        """permite generar estados para entrenar una red. devuelve una matriz lista para ser usada para calcular W"""

        N =self.N # cantidad de neuronas

        rng = np.random.default_rng() # generador de números aleatorios 0 o 1
        vector = np.asarray(rng.integers(2, size=N)).reshape(-1, 1) # generamos 1 tirada de cierto tamaño

        vector = (vector.reshape(-1, 1))# Convertir a columna

        return vector *2 -1 # de -1 a 1
    

    def calcular_W(self, patrones, eta = 1):
        """
        Para calcular la W correspondiente a los patrones recibidos. Se asume que el formato es el que siempre se viene trabajando de vectores colmunas
        Se supone un "eta" unitario por comodidad. 
        """

        n_neuronas = self.N
        n_patrones = patrones.shape[1]

        X = patrones

        W = (X @ X.T - n_patrones*np.eye(n_neuronas)) * eta

        self.W = W

        return
    

    
    
    def step_red_neuronal(self, patron_inicial):
        """
        Patrón inicial debe ser vector columna.
        """
        estado = np.copy(patron_inicial)
        estado = self.W @ estado
        estado = np.sign(estado)
        estado = np.where(estado == 0, 1, estado)  # Manejar ceros
        return estado
        

    def agregar_columna(self, datos,correlacion):
        """Esto lo uso para agregar de a 1 patron a la vez y no tener que hacer muchos randoms"""
        rnd = self.estado_aleatorio(correlacion=correlacion)
        datos = np.hstack((datos, rnd))
        return datos,rnd # debería ser cómodo para cuando itere para encontrar cuando fallan las cosas
    
    def comprobar_memoria(self, original):
        salida = self.step_red_neuronal(original)
        cant_bits_erroneos = np.sum(np.abs(original-salida)/2) # si hago la diferencia y divido por 2 debería obtener 
        # la cantidad de bits diferentes porque 1+1 = 2 ,1-1 = 0 ,-1-1 = -2
        return cant_bits_erroneos
    
    def actualizar_W(self, nuevo_patron, eta=1):
        """
        Actualiza la matriz de pesos de una red de Hopfield con un nuevo patrón. para optimizar un poco
        
        Parámetros
        ----------
        W_vieja : np.ndarray
            Matriz de pesos ya entrenada (N x N).
        nuevo_patron : np.ndarray
            Patrón nuevo en forma de vector columna (N x 1), con valores en {-1, +1}.
        eta : float
            Factor de aprendizaje (default=1).
            
        Retorna
        -------
        np.ndarray
            Nueva matriz de pesos W actualizada.
        """
        W_vieja = self.W
        n_neuronas = W_vieja.shape[0]
        x = nuevo_patron.reshape((n_neuronas, 1))

        # Hebb incremental con eliminación de autoconexiones
        W_nueva = W_vieja + eta * (x @ x.T - np.eye(n_neuronas))

        self.W = W_nueva
        return

    def estimar_errores_vs_patrones(self, max_patron = -1, correlacion = 0):
        if correlacion == 0:
            lista_errores,lista_cant_patrones = _estimar_errores_vs_patrones_descorr(max_patron)
        else:
            lista_errores,lista_cant_patrones = _estimar_errores_vs_patrones_correlacionados(max_patron,correlacion)



        return lista_errores,lista_cant_patrones
    

    def _estimar_errores_vs_patrones_correlacionados(self, max_patron = -1, correlacion):
        if max_patron == -1:
            max_patron = self.N

        lista_cant_patrones = []
        lista_errores = []

        for i in range(max_patron):
            if i == 0: 
                datos = self.estado_aleatorio()
                self.calcular_W(datos) # la mete en el self
            else:
                datos,rnd = self.agregar_columna(datos) # rnd es el nuevo estado
                self.actualizar_W(rnd)


            # acá tenemos una matriz que va a ir aumentando en cantidad de patrones con las iteraciones
            # ahora invoco el cálculo de W
            

            # ahora lo que quiero es iterar por la cantidad de patrones en i y sumar la cantidad de errores
            errores_totales_bits = 0
            for k in range(i):
                estado_original_actual = datos[:,k] # el patrón k-ésimo
                error_actual = self.comprobar_memoria(estado_original_actual) # vamos a ir sumando
                error_actual = error_actual/(self.N * (i+1))
                errores_totales_bits = errores_totales_bits+error_actual

            lista_errores.append(errores_totales_bits)
            lista_cant_patrones.append(i+1)

        return lista_errores,lista_cant_patrones
    

    def _estimar_errores_vs_patrones_descorr(self, max_patron = -1):
        if max_patron == -1:
            max_patron = self.N

        lista_cant_patrones = []
        lista_errores = []

        for i in range(max_patron):
            if i == 0: 
                datos = self.estado_aleatorio()
                self.calcular_W(datos) # la mete en el self
            else:
                datos,rnd = self.agregar_columna(datos) # rnd es el nuevo estado
                self.actualizar_W(rnd)


            # acá tenemos una matriz que va a ir aumentando en cantidad de patrones con las iteraciones
            # ahora invoco el cálculo de W
            

            # ahora lo que quiero es iterar por la cantidad de patrones en i y sumar la cantidad de errores
            errores_totales_bits = 0
            for k in range(i):
                estado_original_actual = datos[:,k] # el patrón k-ésimo
                error_actual = self.comprobar_memoria(estado_original_actual) # vamos a ir sumando
                error_actual = error_actual/(self.N * (i+1))
                errores_totales_bits = errores_totales_bits+error_actual

            lista_errores.append(errores_totales_bits)
            lista_cant_patrones.append(i+1)

        return lista_errores,lista_cant_patrones
    

    def capacidad_dada_proba(lista_errores, lista_cant_patrones, prob_error_max):
        """
        Dada una curva de errores vs patrones, devuelve la máxima cantidad 
        de patrones que se pueden almacenar sin superar un error dado.
        
        Parámetros
        ----------
        lista_errores : list[float]
            Lista de probabilidades de error acumuladas (salida de estimar_errores_vs_patrones).
        lista_cant_patrones : list[int]
            Lista con la cantidad de patrones correspondientes.
        prob_error_max : float
            Probabilidad máxima de error permitida (ej: 0.05).
            
        Retorna
        -------
        int
            Cantidad máxima de patrones que cumple la condición.
        """
        capacidad = 0
        for err, cant in zip(lista_errores, lista_cant_patrones):
            if err <= prob_error_max:
                capacidad = cant
            else:
                break
        return capacidad
    

    def generar_patrones_correlacionados(self, cantidad, correlacion):
        """
        Genera 'cantidad' patrones binarios (-1,1) con una correlación aproximada 'correlacion' respecto a un patrón base.
        correlacion: valor entre 0 (totalmente aleatorio) y 1 (idéntico al patrón base).
        """
        N = self.N
        rng = np.random.default_rng()
        patron_base = rng.choice([-1, 1], size=(N, 1))
        patrones = [patron_base]
        p = (1 - correlacion) / 2
        for _ in range(cantidad - 1):
            ruido = rng.random((N, 1)) < p
            nuevo_patron = np.where(ruido, -patron_base, patron_base)
            patrones.append(nuevo_patron)
        return np.hstack(patrones)


para la primer prueba voy a probar con 1000 neuronas.

In [3]:
ERN = evaluacion_capacidad_red_neuronal(100) 

In [4]:
n_iters = 100
errores, n_patrones = ERN.estimar_errores_vs_patrones()
errores = np.array(errores)
for _ in range(n_iters-1):
    err2,_ = ERN.estimar_errores_vs_patrones()
    errores += np.array(err2)
errores = errores / 50

In [5]:
umbrales =  [0.001,0.0036,0.01,0.05,0.1]
n_neuronas = 50
for umbral in umbrales:
    cap = evaluacion_capacidad_red_neuronal.capacidad_dada_proba(errores, n_patrones, umbral)
    print(f"Para el umbral {umbral} se estima una capacidad de {cap/n_neuronas:.3f}\n")

Para el umbral 0.001 se estima una capacidad de 0.200

Para el umbral 0.0036 se estima una capacidad de 0.260

Para el umbral 0.01 se estima una capacidad de 0.320

Para el umbral 0.05 se estima una capacidad de 0.540

Para el umbral 0.1 se estima una capacidad de 0.760

